In [44]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
from PIL import Image
from torchvision import models, transforms,datasets
from torch.utils.data import DataLoader,Dataset

torch.manual_seed(42) 
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42) 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [62]:
class TestDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.images = [os.path.join(img_dir, img) for img in os.listdir(img_dir)]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        if self.transform: image = self.transform(image)
        return image

In [70]:
def DataPreprocess(train_PATH,val_PATH,test_PATH,batch_size):
    transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.ToTensor(),])
    
    train_data=datasets.ImageFolder(root=train_PATH, transform=transform)
    val_data=datasets.ImageFolder(root=val_PATH, transform=transform)
    test_data=TestDataset(img_dir=test_PATH, transform=transform)

    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
    val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=8)
    test_dataloader = DataLoader(test_data, batch_size=len(test_data), shuffle=False, num_workers=8)
    
    # 0:cat,1:dot 
    # print(train_data.class_to_idx)
    # print(val_data.class_to_idx)
    return train_dataloader,val_dataloader,test_dataloader

In [78]:
class ConvNet(nn.Module):
    def __init__(self,num_classes):
        super(ConvNet, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.fc1 = nn.Linear(in_features=16 * 64 * 64, out_features=256)  # 假设输入图像大小为32x32
        self.fc2 = nn.Linear(in_features=256, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=num_classes)
        
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        # 展平特征图
        x = x.view(-1, 16 * 64 * 64)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x

def train(model,epochs,train_loader,val_loader,test_loader,optimizer,criterion,):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')
        if epoch+1==epochs:break
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy on val set: {100 * correct / total}%')

    model.eval()
    pred=[]
    with torch.no_grad():
        for images in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            pred.extend(predicted.cpu().numpy())
    for _ in range(len(pred)):pred[_]='cat' if _==0 else 'dog'
    res=pd.DataFrame(columns=['label'],data=pred)
    res.to_csv('./res.csv')    
    torch.save(model,'./model.pth')

In [79]:
model=ConvNet(num_classes=2).to(device)

epoch=10
batch_size = 32
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loader,val_loader,test_loader=DataPreprocess('./datasets/train','./datasets/val','./datasets/test',batch_size)
train(model,epoch,train_loader,val_loader,test_loader,optimizer,criterion,)

Epoch 1, Loss: 0.6756611958026886
Accuracy on val set: 66.26%
Epoch 2, Loss: 0.596325344991684
Accuracy on val set: 71.46%
Epoch 3, Loss: 0.5124380988121032
Accuracy on val set: 74.02%
Epoch 4, Loss: 0.41170547456741335
Accuracy on val set: 74.8%
Epoch 5, Loss: 0.31637397929430006
Accuracy on val set: 75.22%
Epoch 6, Loss: 0.22639767491817475
Accuracy on val set: 76.02%
Epoch 7, Loss: 0.16430695915222168
Accuracy on val set: 74.56%
Epoch 8, Loss: 0.12599979072660208
Accuracy on val set: 74.98%
Epoch 9, Loss: 0.11218254924565553
Accuracy on val set: 75.02%
Epoch 10, Loss: 0.09539661161750555
